In [1]:
import random
import pandas as pd
import numpy as np
import implicit
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
import json

In [21]:
raw_data = pd.read_json("./data/part-aa",lines=True,encoding="utf-8")
raw_data.head()

,time,user_id,product_id,belong_cate_lvl1_id,belong_cate_lvl2_id,belong_cate_lvl3_id,belong_cate_lvl1_name,belong_cate_lvl2_name,belong_cate_lvl3_name,href
0,2016-03-31 00:00:00,7892FA71D8D1EAEBE438135718A5C420,2920660,8,9,11,Thời trang nữ,Áo nữ,Áo kiểu,https://www.sendo.vn/san-pham/ao-kieu-from-dai...
1,2016-03-31 00:00:06,2CEBC824EBEF1B52EA7562B716D020D8,1885579,1722,1726,1748,Túi xách,Túi xách nam,Túi đeo chéo nam,https://www.sendo.vn/san-pham/tui-deo-may-tinh...
2,2016-03-31 00:00:07,81ADFE4DC80154EAF829FC0CBADD71EA,2633702,8,52,53,Thời trang nữ,"Đồ lót, đồ ngủ và đồ mặc nhà",Áo ngực,https://www.sendo.vn/san-pham/ao-nguc-7-kieu-c...
3,2016-03-31 00:00:13,F942D7B0377D64F1280EA7BB8DCC3A47,2936475,528,529,542,Phụ kiện công nghệ,Phụ kiện Điện thoại,Phụ kiện khác,https://www.sendo.vn/san-pham/kinh-thuc-te-ao-...
4,2016-03-31 00:00:16,C8BFABB5F293BC9890124E192FBFC898,2146364,8,52,56,Thời trang nữ,"Đồ lót, đồ ngủ và đồ mặc nhà",Đồ lót bộ,https://www.sendo.vn/san-pham/set-ao-yem-quan-...


In [4]:
filename = "part-ac"
chunks = pd.read_json("./data/{}".format(filename),lines=True,encoding="utf-8", chunksize=400000)
part=1
for chunk in chunks:
    print(len(chunk))
    chunk.to_csv("./data/{}_{}.csv".format(filename,part), index=False)
    part+=1

400000
400000
400000
400000
400000
304103


In [22]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 10 columns):
time                     object
user_id                  object
product_id               int64
belong_cate_lvl1_id      int64
belong_cate_lvl2_id      int64
belong_cate_lvl3_id      int64
belong_cate_lvl1_name    object
belong_cate_lvl2_name    object
belong_cate_lvl3_name    object
href                     object
dtypes: int64(4), object(6)
memory usage: 183.1+ MB


In [23]:
raw_data.describe().T

,count,mean,std,min,25%,50%,75%,max
product_id,2400000.0,2.493566e+06,555137.171045,8422.0,2249812.0,2687518.0,2894347.0,3098875.0
belong_cate_lvl1_id,2400000.0,5.420875e+02,675.010478,8.0,8.0,94.0,1366.0,1722.0
belong_cate_lvl2_id,2400000.0,7.226362e+02,689.173266,9.0,26.0,530.0,1394.0,1868.0
belong_cate_lvl3_id,2400000.0,8.671546e+02,652.363948,10.0,129.0,699.0,1456.0,1880.0


In [20]:
raw_data.to_csv("./data/data.csv", index=False)

In [19]:
np.savetxt('test.csv', )

TypeError: savetxt() missing 1 required positional argument: 'X'

In [3]:
raw_data = raw_data.groupby(["user_id","product_id"]).count().reset_index()
raw_data.head()

,user_id,product_id,time,belong_cate_lvl1_id,belong_cate_lvl2_id,belong_cate_lvl3_id,belong_cate_lvl1_name,belong_cate_lvl2_name,belong_cate_lvl3_name,href
0,0009C208AA7A874A5791FF58F4904BA1,2551554,3,3,3,3,3,3,3,3
1,00287A7AE65343DA3028BE1518910877,1348373,1,1,1,1,1,1,1,1
2,002A34CB240ED93E8DDECDB800C717B5,2344633,1,1,1,1,1,1,1,1
3,00492A3DF96D01889EA3A13D0DEDA181,1903157,1,1,1,1,1,1,1,1
4,006C598E51FA386A33570B0205F7F7BF,2437450,1,1,1,1,1,1,1,1


In [4]:
# Drop rows with missing values
data = raw_data[["user_id","product_id","time"]].dropna()

# Convert artists names into numerical IDs
data['user_id_code'] = data['user_id'].astype("category").cat.codes
data['product_id_code'] = data['product_id'].astype("category").cat.codes

In [5]:
product_names=json.load(open("data/docs/product_name_mapping.json","r"))

In [6]:
item_lookup = data[['product_id', 'product_id_code']].drop_duplicates()
item_lookup['product_id_code'] = item_lookup.product_id_code.astype(str)
item_lookup['product_name'] = item_lookup.product_id.apply(lambda x: product_names[str(x)].replace("-"," "))
item_lookup.head()

,product_id,product_id_code,product_name
0,2551554,927,moc khoa kiem katana
1,1348373,150,gn016 giay the thao tre trung
2,2344633,635,giay nike thea ms 131
3,1903157,245,dong ho nam casio efr 535l 1avdf
4,2437450,730,nhan da my inox cao cap bh vinh vien nb043


In [7]:
data = data.drop(['user_id', 'product_id'], axis=1)
data.head()

,time,user_id_code,product_id_code
0,3,0,927
1,1,1,150
2,1,2,635
3,1,3,245
4,1,4,730


In [8]:
users = list(np.sort(data.user_id_code.unique()))
products = list(np.sort(data.product_id_code.unique()))
time = list(data.time)

In [9]:
# Get the rows and columns for our new matrix
rows = data.user_id_code.astype(int)
cols = data.product_id_code.astype(int)

In [10]:
data_sparse = sparse.csr_matrix((time, (rows, cols)), shape=(len(users), len(products)))
sparse_user_item = sparse.csr_matrix((time, (cols, rows)))

In [11]:
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)
alpha_val = 15
data_conf = (data_sparse * alpha_val).astype('double')

#Fit the model
model.fit(data_conf)

In [12]:
len(item_lookup)

2123

In [13]:
item_id = 2 
n_similar = 10

# Use implicit to get similar items.
similar = model.similar_items(item_id, n_similar)

print("product:",item_lookup.product_name.loc[data.product_id_code==item_id].iloc[0])

# Print the names of our most similar artists
for item in similar:
    idx, score = item
    print(raw_data.product_id.loc[data.product_id_code == idx].iloc[0])
    print(item_lookup.product_name.loc[raw_data.product_id.loc[data.product_id_code == idx].iloc[0]==item_lookup.product_id].iloc[0])

product: giay sandal xuat khau nhat ban nv25
247287
giay sandal xuat khau nhat ban nv25
2456942
ao kieu han quoc
2284390
giay tay nam da bo that t64
445748
ban phim apple mini
2316712
dong ho casio efb 300l 7av chinh hang danh cho nam 2683269
2241121
op nokia lumia 620 sgp nham ben dep oln15
2234225
quan tay nam cong so
2475137
do lot cosplay ho 3 vong sexy ql44
1389111
giay da nam dr martens 1461
2338901
quy cotui xach henry thoi trang


In [14]:
user_id = 2000

# Use the implicit recommender.
recommended = model.recommend(user_id, sparse_user_item, N=len(item_lookup), recalculate_user=True)

product_ids = []
scores = []

# Get artist names from ids
for item in recommended:
    idx, score = item
    product_ids.append(item_lookup.product_name.loc[raw_data.product_id.loc[data.product_id_code == idx].iloc[0]==item_lookup.product_id].iloc[0])
    scores.append(score)

# Create a dataframe of artist names and scores

recommendations = pd.DataFrame({'product': product_ids, 'score': scores})


print (recommendations)

                                             product     score
0                               giay the thao ls0891  0.067265
1                                 quan lot nu uniqlo  0.054395
2                   tui xach chanel classic f1 chuan  0.049489
3                     bo gen toan than sexy tb 24713  0.037868
4                                  ao so mi nam f348  0.036700
..                                               ...       ...
853              ao so mi co duc phong cach han quoc -0.038736
854                             dong ho casio baby g -0.040554
855  tinker bell kids bo gap ngan 512 ice cream pink -0.041546
856          combo ban phim va chuot apple khong day -0.042863
857                                     quan lot nam -0.048139

[858 rows x 2 columns]


0.0.0.0:1000/user_based/{$user_id}

[
    {
        "item_id":"item_id_1",
        "title": "Item 1",
        "image": image_href
    },
]

Query item_id_x trong database

In [ ]:
{
    "1": [],
    "2": []
}